## 1.1) Crear Base de datos "datos_padron" .

create database datos_padron

### 1.2) Crear la tabla de datos padron_txt con todos los campos del fichero CSV y cargar losdatos mediante el comando LOAD DATA LOCAL INPATH. La tabla tendrá formato texto y tendrá como delimitador de campo el caracter ';' y los campos que en el documento original están encerrados en comillas dobles '"' no deben estar envueltos en estos caracteres en la tabla de Hive (es importante indicar esto utilizando el serde de OpenCSV, si no la importación de las variables que hemos indicado como numéricas fracasará ya que al estar envueltos en comillas los toma como strings) y se deberá omitir la cabecera del fichero de datos al crear la tabla.

create table padron_txt_aux(COD_DISTRITO int, DESC_DISTRITO string, 
    COD_DIST_BARRIO int, DESC_BARRIO string, COD_BARRIO int, 
    COD_DIST_SECCION string, COD_SECCION int, COD_EDAD_INT int, 
    EspanolesHombres int, EspanolesMujeres int, ExtranjerosHombres int, 
    ExtranjerosMujeres int) 
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
    WITH SERDEPROPERTIES ("separatorChar" = "\;", "quoteChar" = "\"")
    tblproperties("skip.header.line.count"="1");
    
load data local inpath '/home/cloudera/Documents/PadronMadridDatosUTF8.csv' into table padron_txt;

create table padron_txt_bien as select cast(cod_distrito as int), desc_distrito, cast(cod_dist_barrio as int), desc_barrio, cast(cod_barrio as int),
cast(cod_dist_seccion as int), cast(cod_seccion as int), cast(cod_edad_int as int), cast(EspanolesHombres as int), cast(EspanolesMujeres as int),
cast(ExtranjerosHombres as int), cast(ExtranjerosMujeres as int) from padron_txt;

## 1.3) Hacer trim sobre los datos para eliminar los espacios innecesarios guardando la tabla resultado como padron_txt_2. (Este apartado se puede hacer creando la tabla con una sentencia CTAS.)


create table padron_txt_2 as select cast(cod_distrito as int), trim(desc_distrito), cast(cod_dist_barrio as int), trim(desc_barrio), cast(cod_barrio as int),
cast(cod_dist_seccion as int), cast(cod_seccion as int), cast(cod_edad_int as int), cast(EspanolesHombres as int), cast(EspanolesMujeres as int),
cast(ExtranjerosHombres as int), cast(ExtranjerosMujeres as int) from padron_txt;

## 1.4) Investigar y entender la diferencia de incluir la palabra LOCAL en el comando LOAD DATA.

que puedes coger directamente el fichero local.

### 1.5) En este momento te habrás dado cuenta de un aspecto importante, los datos nulos de nuestras tablas vienen representados por un espacio vacío y no por un identificador de nulos comprensible para la tabla. Esto puede ser un problema para el tratamiento posterior de los datos. Podrías solucionar esto creando una nueva tabla utiliando sentencias case when que sustituyan espacios en blanco por 0. Para esto primero comprobaremos que solo hay espacios en blanco en las variables numéricas correspondientes a las últimas 4 variables de nuestra tabla (podemos hacerlo con alguna sentencia de HiveQL) y luego aplicaremos las sentencias case when para sustituir por 0 los espacios en blanco. (Pista: es útil darse cuenta de que un espacio vacío es un campo con longitud 0). Haz esto solo para la tabla padron_txt.

create table padron_txt as select COD_DISTRITO, DESC_DISTRITO , COD_DIST_BARRIO , DESC_BARRIO , COD_BARRIO ,COD_DIST_SECCION , COD_SECCION , COD_EDAD_INT,
case when length(espanoleshombres) == 0 then 0 else espanoleshombres end as espanoleshombres,
case when length(espanolesmujeres) == 0 then 0 else espanolesmujeres end as espanolesmujeres,
case when length(extranjeroshombres) == 0 then 0 else extranjeroshombres end as extranjeroshombres,
case when length(extranjerosmujeres) == 0 then 0 else extranjerosmujeres end as extranjerosmujeres
from padron_txt_aux;

create table padron_txt_1_bien as select cast(cod_distrito as int), trim(desc_distrito), cast(cod_dist_barrio as int), trim(desc_barrio), cast(cod_barrio as int),
cast(cod_dist_seccion as int), cast(cod_seccion as int), cast(cod_edad_int as int), cast(EspanolesHombres as int), cast(EspanolesMujeres as int),
cast(ExtranjerosHombres as int), cast(ExtranjerosMujeres as int) from padron_txt;

### 1.6) Una manera tremendamente potente de solucionar todos los problemas previos (tanto las comillas como los campos vacíos que no son catalogados como null y los espacios innecesarios) es utilizar expresiones regulares (regex) que nos proporciona OpenCSV.Para ello utilizamos :ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' WITH SERDEPROPERTIES ('input.regex'='XXXXXXX') Donde XXXXXX representa una expresión regular que debes completar y que identifique el formato exacto con el que debemos interpretar cada una de las filas de nuestro CSV de entrada. Para ello puede ser útil el portal "regex101". Utiliza este método para crear de nuevo la tabla padron_txt_2

create table if not exists padron_txt1(
COD_DISTRITO int,
DESC_DISTRITO string,
COD_DIST_BARRIO int,
DESC_BARRIO string,
COD_BARRIO int,
COD_DIST_SECCION int,
COD_SECCION int,
COD_EDAD_INT int,
EspanolesHombres int,
EspanolesMujeres int,
ExtranjerosHombres int,
ExtranjerosMujeres int)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES ('input.regex'='"(.*)";"([A-Za-z]*) *";"(.*)";"([A-Za-z]*) *";"(.*)";"(.*?)";"(.*?)";"(.*?)";"(.*?)";"(.*?)";"(.*?)";"(.*?)"')
stored as textfile
tblproperties("skip.header.line.count"="1");

load data local inpath '/home/cloudera/Documents/PadronMadridDatosUTF8.csv' into table padron_txt1;

create table padron_txt_2 as select COD_DISTRITO, DESC_DISTRITO , COD_DIST_BARRIO , DESC_BARRIO , COD_BARRIO ,COD_DIST_SECCION , COD_SECCION , COD_EDAD_INT,
case when length(espanoleshombres) == 0 then '0' else espanoleshombres end as espanoleshombres,
case when length(espanolesmujeres) == 0 then '0' else espanolesmujeres end as espanolesmujeres,
case when length(extranjeroshombres) == 0 then '0' else extranjeroshombres end as extranjeroshombres,
case when length(extranjerosmujeres) == 0 then '0' else extranjerosmujeres end as extranjerosmujeres
from padron_txt1;

## 2.1)¿Qué es CTAS?

Create table as

## 2.2) Crear tabla Hive padron_parquet (cuyos datos serán almacenados en el formato columnar parquet) a través de la tabla padron_txt mediante un CTAS.

CREATE TABLE padron_parquet stored as parquet
AS select * from padron_txt_bien;

## 2.3) Crear tabla Hive padron_parquet_2 a través de la tabla padron_txt_2 mediante un CTAS. En este punto deberíamos tener 4 tablas, 2 en txt (padron_txt y padron_txt_2, la primera con espacios innecesarios y la segunda sin espacios innecesarios) y otras dos tablas en formato parquet (padron_parquet y padron_parquet_2, la primera con espacios y la segunda sin ellos).

CREATE TABLE padron_parquet_2 stored as parquet
AS select * from padron_txt_2;

## 2.5) Investigar en qué consiste el formato columnar parquet y las ventajas de trabajar con este tipo de formatos.

Es un compresor de datos

## 2.6) Comparar el tamaño de los ficheros de los datos de las tablas padron_txt (txt), padron_txt_2 (txt pero no incluye los espacios innecesarios), padron_parquet y padron_parquet_2 (alojados en hdfs cuya ruta se puede obtener de la propiedad location de cada tabla por ejemplo haciendo "show create table").

padron_txt: 16.170 MB
padron_txt_2: 9.3678 MB
padron_parquet: 911.16 KB
padron_parquet_2: 369.85 KB

## 3.1) ¿Qué es Impala?

Motor de consultas de SQL para big data

## 3.2) ¿En qué se diferencia de Hive?

Impala es más rápido pero no tolera fallo

## 3.3) Comando INVALIDATE METADATA, ¿en qué consiste?

Cuando impala vaya a utilizar una tabla cuya metadata ha sido cambiada, impala actualizará la metadata antes de utilizar la tabla

## 3.4)Hacer invalidate metadata en Impala de la base de datos datos_padron

invalidate metadata;

## 3.5) Calcular el total de EspanolesHombres, espanolesMujeres, ExtranjerosHombres y ExtranjerosMujeres agrupado por DESC_DISTRITO y DESC_BARRIO.
## 3.6) Llevar a cabo las consultas en Hive en las tablas padron_txt_2 y padron_parquet_2 (No deberían incluir espacios innecesarios). ¿Alguna conclusión?

select count(EspanolesHombres), count(espanolesMujeres), count(ExtranjerosHombres) ,count(ExtranjerosMujeres)
 from padron_txt_2
 group by  DESC_DISTRITO ,DESC_BARRIO;

select count(EspanolesHombres), count(espanolesMujeres), count(ExtranjerosHombres) ,count(ExtranjerosMujeres)
 from padron_parquet_2
 group by  DESC_DISTRITO ,DESC_BARRIO;

## 3.7) Llevar a cabo la misma consulta sobre las mismas tablas en Impala. ¿Alguna conclusión?
## 3.8) ¿Se percibe alguna diferencia de rendimiento entre Hive e Impala?

mucho mas rapido en impala

## 4.1) Crear tabla (Hive) padron_particionado particionada por campos DESC_DISTRITO y DESC_BARRIO cuyos datos estén en formato parquet.

create table padron_particionado(COD_DISTRITO int,COD_DIST_BARRIO int,COD_BARRIO int,COD_DIST_SECCION int,COD_SECCION int,COD_EDAD_INT int,EspanolesHombres int,EspanolesMujeres int,ExtranjerosHombres int,ExtranjerosMujeres int)
partitioned by (desc_distrito string, desc_barrio string)
STORED AS parquet;

## 4.2) Insertar datos (en cada partición) dinámicamente (con Hive) en la tabla recién creada a partir de un select de la tabla padron_parquet_2.

FROM datos_padron.padron_parquet_2
INSERT OVERWRITE TABLE datos_padron.padron_particionado
PARTITION(DESC_DISTRITO, DESC_BARRIO)
SELECT COD_DISTRITO, COD_DIST_BARRIO, COD_BARRIO, COD_DIST_SECCION, COD_SECCION, COD_EDAD_INT,
EspanolesHombres,EspanolesMujeres, ExtranjerosHombres,ExtranjerosMujeres, DESC_DISTRITO, DESC_BARRIO;

CREATE TABLE datos_padron.padron_particionado(COD_DISTRITO INT, COD_DIST_BARRIO INT, 
                                 COD_BARRIO INT, COD_DIST_SECCION INT,
                                 COD_SECCION INT, COD_EDAD_INT INT,
                                 EspanolesHombres INT, EspanolesMujeres INT,
                                 ExtranjerosHombres INT, ExtranjerosMujeres INT)

PARTITIONED BY(DESC_DISTRITO STRING, DESC_BARRIO STRING)

STORED AS PARQUET;


SET hive.exec.dynamic.partition=true;
SET hive.exec.dynamic.partition.mode=non-strict;
SET hive.exec.max.dynamic.partitions = 10000;
SET hive.exec.max.dynamic.partitions.pernode = 1000;

SET mapreduce.map.memory.mb = 2048;
SET mapreduce.reduce.memory.mb = 2048;
SET mapreduce.map.java.opts=-Xmx1800m;

FROM datos_padron.padron_parquet2

INSERT OVERWRITE TABLE datos_padron.padron_particionado

PARTITION(DESC_DISTRITO, DESC_BARRIO)

SELECT COD_DISTRITO, COD_DIST_BARRIO, COD_BARRIO, COD_DIST_SECCION, COD_SECCION, COD_EDAD_INT, 
       EspanolesHombres,EspanolesMujeres, ExtranjerosHombres,ExtranjerosMujeres, DESC_DISTRITO, DESC_BARRIO;


## 4.4) Calcular el total de EspanolesHombres, EspanolesMujeres, ExtranjerosHombres y ExtranjerosMujeres agrupado por DESC_DISTRITO y DESC_BARRIO para los distritos CENTRO, LATINA, CHAMARTIN, TETUAN, VICALVARO y BARAJAS.
## 4.5) Llevar a cabo la consulta en Hive en las tablas padron_parquet y padron_partitionado. ¿Alguna conclusión?

select sum(espanoleshombres), sum(espanolesmujeres), sum(extranjeroshombres), sum(extranjerosmujeres) 
from padron_parquet where desc_distrito in ('CENTRO','LATINA','CHAMARTIN','TETUAN','VICALVARO','BARAJAS')
group by desc_distrito, desc_barrio;

select sum(espanoleshombres), sum(espanolesmujeres), sum(extranjeroshombres), sum(extranjerosmujeres) 
from padron_particionado where desc_distrito in ('CENTRO','LATINA','CHAMARTIN','TETUAN','VICALVARO','BARAJAS')
group by desc_distrito, desc_barrio;

El parquet no devuelve ningun valor porque tiene espacios

## 4.6)Llevar a cabo la consulta en Impala en las tablas padron_parquet y padron_particionado. ¿Alguna conclusión?

impala es mucho mas rapido

## 4.7) Hacer consultas de agregación (Max, Min, Avg, Count) tal cual el ejemplo anterior con las 3 tablas (padron_txt_2, padron_parquet_2 y padron_particionado) y comparar rendimientos tanto en Hive como en Impala y sacar conclusiones.

select max(espanoleshombres), min(espanolesmujeres), count(extranjeroshombres), sum(extranjerosmujeres) 
from padron_txt_2 where desc_distrito in ('CENTRO','LATINA','CHAMARTIN','TETUAN','VICALVARO','BARAJAS')
group by desc_distrito, desc_barrio;

select max(espanoleshombres), min(espanolesmujeres), count(extranjeroshombres), sum(extranjerosmujeres) 
from padron_particionado where desc_distrito in ('CENTRO','LATINA','CHAMARTIN','TETUAN','VICALVARO','BARAJAS')
group by desc_distrito, desc_barrio;

select max(espanoleshombres), min(espanolesmujeres), count(extranjeroshombres), sum(extranjerosmujeres) 
from padron_parquet_2 where desc_distrito in ('CENTRO','LATINA','CHAMARTIN','TETUAN','VICALVARO','BARAJAS')
group by desc_distrito, desc_barrio;

impala parquet2: 0s
impala particionado: 0s
impala txt2: 0s
hive parquet2: 1 41s
hive particionado: 1m 39s
hive txt2: 1m 48s

## 5.1) Crear un documento de texto en el almacenamiento local que contenga una secuencia de números distribuidos en filas y separados por columnas, llámalo datos1 y que sea por ejemplo:
    1,2,3
    4,5,6
    7,8,9
## 5.2) Crear un segundo documento (datos2) con otros números pero la misma estructura

### 5.3) Crear un directorio en HDFS con un nombre a placer, por ejemplo, /test. Si estás en una máquina Cloudera tienes que asegurarte de que el servicio HDFS está activo ya que puede no iniciarse al encender la máquina (puedes hacerlo desde el Cloudera Manager). A su vez, en las máquinas Cloudera es posible (dependiendo de si usamos Hive desde consola o desde Hue) que no tengamos permisos para crear directorios en HDFS salvo en el directorio /user/cloudera

## 5.4) Mueve tu fichero datos1 al directorio que has creado en HDFS con un comando desde consola.

mkdir \test
cp datos1 \test

## 5.5) Desde Hive, crea una nueva database por ejemplo con el nombre numeros. Crea una tabla que no sea externa y sin argumento location con tres columnas numéricas, campos separados por coma y delimitada por filas. La llamaremos por ejemplo numeros_tbl.

create database numeros;
use numeros;

create table numeros_tbl(num1 int,num2 int, num3 int)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
    WITH SERDEPROPERTIES ("separatorChar" = ",");

## 5.6) Carga los datos de nuestro fichero de texto datos1 almacenado en HDFS en la tabla de Hive. Consulta la localización donde estaban anteriormente los datos almacenados. ¿Siguen estando ahí? ¿Dónde están?. Borra la tabla, ¿qué ocurre con los datos almacenados en HDFS?

load data inpath '/test/datos1' into table numeros_tbl;

los datos han desaparecido, se han ido al warehouse
si borro la table entonces tambien desaparece del warehouse

## 5.7) Vuelve a mover el fichero de texto datos1 desde el almacenamiento local al directorio anterior en HDFS.
## 5.8)Desde Hive, crea una tabla externa sin el argumento location. Y carga datos1 (desde HDFS) en ella. ¿A dónde han ido los datos en HDFS? Borra la tabla ¿Qué ocurre con los datos en hdfs?

create external table numeros_tbl(num1 int,num2 int, num3 int)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
    WITH SERDEPROPERTIES ("separatorChar" = ",");
    
se han ido tambien al warehouse pero luego no desaparece

### 5.9) Borra el fichero datos1 del directorio en el que estén. Vuelve a insertarlos en el directorio que creamos inicialmente (/test). Vuelve a crear la tabla numeros desde hive pero ahora de manera externa y con un argumento location que haga referencia al directorio donde los hayas situado en HDFS (/test). No cargues los datos de ninguna manera explícita. Haz una consulta sobre la tabla que acabamos de crear que muestre todos los registros. ¿Tiene algún contenido?

create external table numeros_tbl_external_loc(num1 int,num2 int, num3 int)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES ("separatorChar" = ",")   
LOCATION '/test';

select * from numeros_tbl_external_loc;

## 5.10) Inserta el fichero de datos creado al principio, "datos2" en el mismo directorio de HDFS que "datos1". Vuelve a hacer la consulta anterior sobre la misma tabla. ¿Qué salida muestra? 


se une la segunda tabla por debajo de la primera

## 5.11) Extrae conclusiones de todos estos anteriores apartados.

al cargar los datos se borran 
para location hace falta tener unicamente los datos en la localizacion

## 6.1) Comenzamos realizando la misma práctica que hicimos en Hive en Spark, importando el csv. Sería recomendable intentarlo con opciones que quiten las "" de los campos, que ignoren los espacios innecesarios en los campos, que sustituyan los valores vacíos por 0 y que infiera el esquema

In [4]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("padron")
                        .master("local")
                        .enableHiveSupport()
                        .getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5563efbc


In [3]:
spark.stop()

In [4]:
val df = spark.read.format("csv")
            .option("header", "true")
            .option("inferSchema", "true")
            .option("delimiter",";")
            .option("quote","\"")
            .option("escape","\"")
            .load("C:/Users/xabier.jimenez/Downloads/PadronMadridDatosUTF8.csv")
            .na.fill(0).cache()
df.show(5)

+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|       DESC_DISTRITO|COD_DIST_BARRIO|         DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|           1|CENTRO              |            101|PALACIO             |         1|            1001|          1|           0|               2|               3|                 1|                 0|
|           1|CENTRO              |            101|PALACIO             |         1|            1001|          1|           1|               7|               0|                 1|                 0|
|         

df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


## 6.2) De manera alternativa también se puede importar el csv con menos tratamiento en la importación y hacer todas las modificaciones para alcanzar el mismo estado de limpieza de los datos con funciones de Spark.

In [4]:
val df2 = spark.read.format("csv")
            .option("header", "true")
            .option("inferSchema", "true")
            .option("delimiter",";")
            .load("C:/Users/xabier.jimenez/Downloads/PadronMadridDatosUTF8.csv")
            .withColumn("DESC_DISTRITO", trim(col("DESC_DISTRITO")))
            .withColumn("DESC_BARRIO", trim(col("DESC_BARRIO")))
            .na.fill(0).cache()
df2.show(5)

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               2|               3|                 1|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           1|               7|               0|                 1|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|      

df2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


## 6.3) Enumera todos los barrios diferentes.

In [6]:
df2.select(col("desc_barrio")).distinct().show(15)

+--------------------+
|         desc_barrio|
+--------------------+
|        VALDEFUENTES|
|            ABRANTES|
|       LOS JERONIMOS|
|            VALVERDE|
|              CORTES|
|   PALOMERAS SURESTE|
|CIUDAD UNIVERSITARIA|
|      CUATRO VIENTOS|
|           TRAFALGAR|
|              HELLIN|
|    ALAMEDA DE OSUNA|
|          PRADOLONGO|
|            MOSCARDO|
|          VALDEZARZA|
|           RECOLETOS|
+--------------------+
only showing top 15 rows



##  6.4)Crea una vista temporal de nombre "padron" y a través de ella cuenta el número de barrios diferentes que hay.

In [7]:
df2.createOrReplaceTempView("padron")
spark.sql("""SELECT count(distinct COD_DIST_BARRIO) from padron""").show()

+-------------------------------+
|count(DISTINCT COD_DIST_BARRIO)|
+-------------------------------+
|                            131|
+-------------------------------+



## 6.5) Crea una nueva columna que muestre la longitud de los campos de la columna DESC_DISTRITO y que se llame "longitud"

In [8]:
val df2len = df2.withColumn("longitud",length(col("DESC_DISTRITO")))
df2len.show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|longitud|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               2|               3|                 1|                 0|       6|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           1|               7|               0|                 1|                 0|       6|
|           1|       CENTRO|            101| 

df2len: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]


## 6.6) Crea una nueva columna que muestre el valor 5 para cada uno de los registros de la tabla.

In [9]:
val df2len5 = df2len.withColumn("5",lit(5))
df2len5.show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+---+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|longitud|  5|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+---+
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               2|               3|                 1|                 0|       6|  5|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           1|               7|               0|                 1|                 0|       6|  5|
|           1|       CENT

df2len5: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 12 more fields]


## 6.7)Borra esta columna.

In [10]:
val df2len5borrado = df2len5.drop("5")
df2len5borrado.show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|longitud|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               2|               3|                 1|                 0|       6|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           1|               7|               0|                 1|                 0|       6|
|           1|       CENTRO|            101| 

df2len5borrado: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]


## 6.8) Particiona el DataFrame por las variables DESC_DISTRITO y DESC_BARRIO

In [11]:
val dfpart=df2.repartition($"DESC_DISTRITO", $"DESC_BARRIO")

dfpart: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


## 6.9) Almacénalo en caché. Consulta en el puerto 4040 (UI de Spark) de tu usuario local el estado de los rdds almacenados.

In [12]:
dfpart.cache()

res8: dfpart.type = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


### 6.10)Lanza una consulta contra el DF resultante en la que muestre el número total de "espanoleshombres", "espanolesmujeres", extranjeroshombres" y "extranjerosmujeres" para cada barrio de cada distrito. Las columnas distrito y barrio deben ser las primeras en aparecer en el show. Los resultados deben estar ordenados en orden de más a menos según la columna "extranjerosmujeres" y desempatarán por la columna "extranjeroshombres".

In [13]:
val dfpart2 = dfpart.groupBy("DESC_BARRIO","DESC_DISTRITO")
        .agg(sum("espanoleshombres").alias("espanoleshombres"),sum("espanolesmujeres").alias("espanolesmujeres"),
             sum("extranjeroshombres").alias("extranjeroshombres"),sum("extranjerosmujeres").alias("extranjerosmujeres"))
        .orderBy(desc("extranjerosmujeres"),desc("extranjeroshombres")).show(5)

+------------+------------------+----------------+----------------+------------------+------------------+
| DESC_BARRIO|     DESC_DISTRITO|espanoleshombres|espanolesmujeres|extranjeroshombres|extranjerosmujeres|
+------------+------------------+----------------+----------------+------------------+------------------+
|   SAN DIEGO|PUENTE DE VALLECAS|           13885|           15555|              6871|              7049|
|      ALUCHE|            LATINA|           24928|           29666|              5425|              6313|
|PUEBLO NUEVO|     CIUDAD LINEAL|           23255|           27261|              5558|              6291|
|VISTA ALEGRE|       CARABANCHEL|           15926|           19377|              5496|              6143|
| EMBAJADORES|            CENTRO|           16640|           16593|              8214|              6037|
+------------+------------------+----------------+----------------+------------------+------------------+
only showing top 5 rows



dfpart2: Unit = ()


## 6.11) Elimina el registro en caché.

In [14]:
dfpart.unpersist()

res9: dfpart.type = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


### 6.12) Crea un nuevo DataFrame a partir del original que muestre únicamente una columna con DESC_BARRIO, otra con DESC_DISTRITO y otra con el número total de "espanoleshombres" residentes en cada distrito de cada barrio. Únelo (con un join) con el DataFrame original a través de las columnas en común.

In [40]:
val dfjoin = df2.groupBy("DESC_BARRIO","DESC_DISTRITO")
.agg(sum("espanoleshombres").alias("espanoleshombres"))
.select("DESC_BARRIO","DESC_DISTRITO","espanoleshombres")

val dfjoin2 = dfjoin.join(df2.as('a),a("DESC_BARRIO") === dfjoin("DESC_BARRIO") && a("DESC_DISTRITO") === dfjoin("DESC_DISTRITO"))
dfjoin2.show(5)

+-----------+-------------+----------------+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|DESC_BARRIO|DESC_DISTRITO|espanoleshombres|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+-----------+-------------+----------------+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|    PALACIO|       CENTRO|            9277|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               2|               3|                 1|                 0|
|    PALACIO|       CENTRO|            9277|           1|       CENTRO|            101|    PALACIO| 

dfjoin: org.apache.spark.sql.DataFrame = [DESC_BARRIO: string, DESC_DISTRITO: string ... 1 more field]
dfjoin2: org.apache.spark.sql.DataFrame = [DESC_BARRIO: string, DESC_DISTRITO: string ... 13 more fields]


## 6.13) Repite la función anterior utilizando funciones de ventana. (over(Window.partitionBy.....))

In [42]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val window  = Window.partitionBy("DESC_BARRIO","DESC_DISTRITO")
df.withColumn("espanoleshombres",sum("espanoleshombres").over(window)).show(5)

+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|       DESC_DISTRITO|COD_DIST_BARRIO|         DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|espanoleshombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|          20|SAN BLAS-CANILLEJAS |           2003|AMPOSTA             |         3|           20031|         31|           0|            3255|               4|                 1|                 2|
|          20|SAN BLAS-CANILLEJAS |           2003|AMPOSTA             |         3|           20031|         31|           1|            3255|               6|                 3|                 2|
|         

import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
window: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2d3a9fd8


### 6.14)Mediante una función Pivot muestra una tabla (que va a ser una tabla de contingencia) que contenga los valores totales ()la suma de valores) de espanolesmujeres para cada distrito y en cada rango de edad (COD_EDAD_INT). Los distritos incluidos deben ser únicamente CENTRO, BARAJAS y RETIRO y deben figurar como columnas . El aspecto debe ser similar a este

In [61]:
val dfpivot = df2.where($"DESC_DISTRITO"==="CENTRO" || $"DESC_DISTRITO"==="BARAJAS" || $"DESC_DISTRITO"==="RETIRO")
                .groupBy("COD_EDAD_INT").pivot("DESC_DISTRITO")
                .agg(sum("espanolesmujeres")).orderBy("COD_EDAD_INT")


dfpivot: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_EDAD_INT: int, BARAJAS: bigint ... 2 more fields]


## 6.15) Utilizando este nuevo DF, crea 3 columnas nuevas que hagan referencia a qué porcentaje de la suma de "espanolesmujeres" en los tres distritos para cada rango de edad representa cada uno de los tres distritos. Debe estar redondeada a 2 decimales. Puedes imponerte la condición extra de no apoyarte en ninguna columna auxiliar creada para el caso.

In [72]:
val dfpivot2 =dfpivot.withColumn("porcBar",round($"BARAJAS"/($"BARAJAS"+$"CENTRO"+$"RETIRO"),2))
                    .withColumn("porCent",round($"CENTRO"/($"BARAJAS"+$"CENTRO"+$"RETIRO"),2))
                    .withColumn("porcRet",round($"RETIRO"/($"BARAJAS"+$"CENTRO"+$"RETIRO"),2)).show(10)


+------------+-------+------+------+-------+-------+-------+
|COD_EDAD_INT|BARAJAS|CENTRO|RETIRO|porcBar|porCent|porcRet|
+------------+-------+------+------+-------+-------+-------+
|           0|    146|   240|   294|   0.21|   0.35|   0.43|
|           1|    199|   243|   346|   0.25|   0.31|   0.44|
|           2|    180|   223|   343|   0.24|    0.3|   0.46|
|           3|    204|   229|   383|   0.25|   0.28|   0.47|
|           4|    231|   228|   419|   0.26|   0.26|   0.48|
|           5|    243|   231|   423|   0.27|   0.26|   0.47|
|           6|    257|   257|   427|   0.27|   0.27|   0.45|
|           7|    244|   241|   435|   0.27|   0.26|   0.47|
|           8|    268|   223|   428|   0.29|   0.24|   0.47|
|           9|    245|   254|   430|   0.26|   0.27|   0.46|
+------------+-------+------+------+-------+-------+-------+
only showing top 10 rows



dfpivot2: Unit = ()


## 6.16) Guarda el archivo csv original particionado por distrito y por barrio (en ese orden) en un directorio local. Consulta el directorio para ver la estructura de los ficheros y comprueba que es la esperada.

In [5]:
df2.write.partitionBy("DESC_DISTRITO", "DESC_BARRIO").format("csv").mode("overwrite").save("/tmp/data/csv/df2_csv")

## 6.17)Haz el mismo guardado pero en formato parquet. Compara el peso del archivo con el resultado anterior

In [9]:
df2.write.partitionBy("DESC_DISTRITO", "DESC_BARRIO").format("parquet").mode("overwrite").save("/tmp/data/parquet/df2_parquetcsv")

org.apache.spark.SparkException:  Job aborted.

### 7.1) Por último, prueba a hacer los ejercicios sugeridos en la parte de Hive con el csv "Datos Padrón" (incluyendo la importación con Regex) utilizando desde Spark EXCLUSIVAMENTE sentencias spark.sql, es decir, importar los archivos desde local directamente como tablasde Hive y haciendo todas las consultas sobre estas tablas sin transformarlas en ningún momento en DataFrames ni DataSets

In [6]:
spark.sql("""create database datos_padron;""")
spark.sql("""use datos_padron;""")

res4: org.apache.spark.sql.DataFrame = []


In [7]:
spark.sql("""create table padron_txt_aux(COD_DISTRITO int, DESC_DISTRITO string, 
    COD_DIST_BARRIO int, DESC_BARRIO string, COD_BARRIO int, 
    COD_DIST_SECCION string, COD_SECCION int, COD_EDAD_INT int, 
    EspanolesHombres int, EspanolesMujeres int, ExtranjerosHombres int, 
    ExtranjerosMujeres int) 
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
    WITH SERDEPROPERTIES ("separatorChar" = "\;", "quoteChar" = "\"")
    tblproperties("skip.header.line.count"="1");""")
    
spark.sql("""load data local inpath 'C:/Users/xabier.jimenez/Downloads/PadronMadridDatosUTF8.csv' into table padron_txt_aux;""")

spark.sql("""create table padron_txt_bien as select cast(cod_distrito as int), desc_distrito, cast(cod_dist_barrio as int), desc_barrio, cast(cod_barrio as int),
cast(cod_dist_seccion as int), cast(cod_seccion as int), cast(cod_edad_int as int), cast(EspanolesHombres as int), cast(EspanolesMujeres as int),
cast(ExtranjerosHombres as int), cast(ExtranjerosMujeres as int) from padron_txt_aux;""")

res5: org.apache.spark.sql.DataFrame = []


In [8]:
spark.sql("""create table padron_txt_2 as select cast(cod_distrito as int), trim(desc_distrito), cast(cod_dist_barrio as int), trim(desc_barrio), cast(cod_barrio as int),
cast(cod_dist_seccion as int), cast(cod_seccion as int), cast(cod_edad_int as int), cast(EspanolesHombres as int), cast(EspanolesMujeres as int),
cast(ExtranjerosHombres as int), cast(ExtranjerosMujeres as int) from padron_txt_bien;""")

res6: org.apache.spark.sql.DataFrame = []


In [11]:
spark.sql("""create table padron_txt as select COD_DISTRITO, DESC_DISTRITO , COD_DIST_BARRIO , DESC_BARRIO , COD_BARRIO ,COD_DIST_SECCION , COD_SECCION , COD_EDAD_INT,
case when length(espanoleshombres) == 0 then 0 else espanoleshombres end as espanoleshombres,
case when length(espanolesmujeres) == 0 then 0 else espanolesmujeres end as espanolesmujeres,
case when length(extranjeroshombres) == 0 then 0 else extranjeroshombres end as extranjeroshombres,
case when length(extranjerosmujeres) == 0 then 0 else extranjerosmujeres end as extranjerosmujeres
from padron_txt_aux;""")

spark.sql("""drop table if exists padron_txt_1_bien""")

spark.sql("""create table padron_txt_1_bien as select cast(cod_distrito as int), desc_distrito, cast(cod_dist_barrio as int), desc_barrio, cast(cod_barrio as int),
cast(cod_dist_seccion as int), cast(cod_seccion as int), cast(cod_edad_int as int), cast(EspanolesHombres as int), cast(EspanolesMujeres as int),
cast(ExtranjerosHombres as int), cast(ExtranjerosMujeres as int) from padron_txt;""")

res9: org.apache.spark.sql.DataFrame = []


In [13]:
spark.sql("""create table if not exists padron_txt1(
COD_DISTRITO int,
DESC_DISTRITO string,
COD_DIST_BARRIO int,
DESC_BARRIO string,
COD_BARRIO int,
COD_DIST_SECCION int,
COD_SECCION int,
COD_EDAD_INT int,
EspanolesHombres int,
EspanolesMujeres int,
ExtranjerosHombres int,
ExtranjerosMujeres int)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES ('input.regex'='"(.*)";"([A-Za-z]*) *";"(.*)";"([A-Za-z]*) *";"(.*)";"(.*?)";"(.*?)";"(.*?)";"(.*?)";"(.*?)";"(.*?)";"(.*?)"')
stored as textfile
tblproperties("skip.header.line.count"="1");""")

spark.sql("""load data local inpath 'C:/Users/xabier.jimenez/Downloads/PadronMadridDatosUTF8.csv' into table padron_txt1;""")

spark.sql("""drop table padron_txt_2""")

spark.sql("""create table padron_txt_2 as select COD_DISTRITO, DESC_DISTRITO , COD_DIST_BARRIO , DESC_BARRIO , COD_BARRIO ,COD_DIST_SECCION , COD_SECCION , COD_EDAD_INT,
case when length(espanoleshombres) == 0 then '0' else espanoleshombres end as espanoleshombres,
case when length(espanolesmujeres) == 0 then '0' else espanolesmujeres end as espanolesmujeres,
case when length(extranjeroshombres) == 0 then '0' else extranjeroshombres end as extranjeroshombres,
case when length(extranjerosmujeres) == 0 then '0' else extranjerosmujeres end as extranjerosmujeres
from padron_txt1;""")

res11: org.apache.spark.sql.DataFrame = []


In [14]:
spark.sql("""select * from padron_txt_1_bien limit 10""").show()

+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|cod_distrito|       desc_distrito|cod_dist_barrio|         desc_barrio|cod_barrio|cod_dist_seccion|cod_seccion|cod_edad_int|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|        null|       DESC_DISTRITO|           null|         DESC_BARRIO|      null|            null|       null|        null|            null|            null|              null|              null|
|           1|CENTRO              |            101|PALACIO             |         1|            1001|          1|           0|               2|               3|                 1|                 0|
|         

In [15]:
spark.sql("""select * from padron_txt_2 limit 10""").show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|espanoleshombres|espanolesmujeres|extranjeroshombres|extranjerosmujeres|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|        null|         null|           null|       null|      null|            null|       null|        null|            null|            null|              null|              null|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               2|               3|                 1|              null|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|      

In [16]:
spark.sql("""CREATE TABLE padron_parquet stored as parquet
AS select * from padron_txt_1_bien;""")

res14: org.apache.spark.sql.DataFrame = []


In [17]:
spark.sql("""CREATE TABLE padron_parquet_2 stored as parquet
AS select * from padron_txt_2;""")

res15: org.apache.spark.sql.DataFrame = []


In [18]:
spark.sql("""select count(EspanolesHombres), count(espanolesMujeres), count(ExtranjerosHombres) ,count(ExtranjerosMujeres)
 from padron_txt_2
 group by  DESC_DISTRITO ,DESC_BARRIO;""").show()

+-----------------------+-----------------------+-------------------------+-------------------------+
|count(EspanolesHombres)|count(espanolesMujeres)|count(ExtranjerosHombres)|count(ExtranjerosMujeres)|
+-----------------------+-----------------------+-------------------------+-------------------------+
|                   1359|                   1450|                      649|                      766|
|                   2351|                   2440|                     1361|                     1518|
|                   2243|                   2316|                      582|                      695|
|                   1077|                   1129|                      681|                      668|
|                   2330|                   2490|                     1358|                     1424|
|                   1546|                   1635|                      940|                     1021|
|                   1184|                   1249|                      424|       